# Calculating Inequality

To get a sense of how inequality in Connecticut's cities compared nationally, we decided to look at the ratio between two benchmarks: The lowest combined income a household could earn while still breaking into the top 5 percent of household income and the highest combined income a household could earn while still falling in the bottom 20 percent. The farther apart these two numbers are, proportionally, the greater the gap between rich and poor.

The US Census Bureau provides estimates of the numbers we need in a table titled [HOUSEHOLD INCOME QUINTILE UPPER LIMITS](https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=ACS_16_5YR_B19080&prodType=table).

In [1]:
import pandas

acs = pandas.read_csv('ACS_16_1YR_B19080-1/ACS_16_1YR_B19080.csv')

## Bottom 20 cutoff is HD01_VD02, top 5 cutoff is HD01_VD06
acs = acs[["GEO.id2","GEO.display-label","HD01_VD02","HD01_VD06"]].rename(columns = {"HD01_VD02":"Upper_Limit_Bottom_20","HD01_VD06":"Lower_Limit_Top_5"})

## We're only interested in the 100 biggest metros, so let's join the resident population table and
## only include the largest.
#
## Link to census: https://factfinder.census.gov/faces/tableservices/jsf/pages/productview.xhtml?pid=PEP_2017_PEPANNRES&prodType=table
#
pop_16 = pandas.read_csv("PEP_2017_PEPANNRES-1/PEP_2017_PEPANNRES.csv",encoding = "ISO-8859-1")
acs = acs.join(pop_16[["GEO.id2","respop72016"]].set_index("GEO.id2"),on="GEO.id2")
acs = acs.sort_values("respop72016",ascending=False).reset_index(drop=True)
acs = acs.loc[0:99]

acs.head(10)

,GEO.id2,GEO.display-label,Upper_Limit_Bottom_20,Lower_Limit_Top_5,respop72016
0,35620,"New York-Newark-Jersey City, NY-NJ-PA Metro Area",25717,NaN,20275179
1,31080,"Los Angeles-Long Beach-Anaheim, CA Metro Area",25626,NaN,13328261
2,16980,"Chicago-Naperville-Elgin, IL-IN-WI Metro Area",25921,245949.0,9546326
3,19100,"Dallas-Fort Worth-Arlington, TX Metro Area",28601,234781.0,7253424
4,26420,"Houston-The Woodlands-Sugar Land, TX Metro Area",25785,NaN,6798010
5,47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV M...",41076,NaN,6150681
6,33100,"Miami-Fort Lauderdale-West Palm Beach, FL Metr...",21198,221668.0,6107433
7,37980,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Me...",25571,246971.0,6077152
8,12060,"Atlanta-Sandy Springs-Roswell, GA Metro Area",26684,234699.0,5795723
9,14460,"Boston-Cambridge-Newton, MA-NH Metro Area",31367,NaN,4805942


#### Problem!
There are values missing from this table! The Census doesn't report estimates higher than $250,000. This is an example of [topcoding](link), which the Census says it does for privacy reasons. 16 of the cities in our dataset have topcoded lower limits to their top 5 percent of income earners.

In [2]:
top_coded = acs[acs["Lower_Limit_Top_5"].isnull()]
top_coded

,GEO.id2,GEO.display-label,Upper_Limit_Bottom_20,Lower_Limit_Top_5,respop72016
0,35620,"New York-Newark-Jersey City, NY-NJ-PA Metro Area",25717,NaN,20275179
1,31080,"Los Angeles-Long Beach-Anaheim, CA Metro Area",25626,NaN,13328261
4,26420,"Houston-The Woodlands-Sugar Land, TX Metro Area",25785,NaN,6798010
5,47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV M...",41076,NaN,6150681
9,14460,"Boston-Cambridge-Newton, MA-NH Metro Area",31367,NaN,4805942
10,41860,"San Francisco-Oakland-Hayward, CA Metro Area",36353,NaN,4699077
14,42660,"Seattle-Tacoma-Bellevue, WA Metro Area",34576,NaN,3802660
16,41740,"San Diego-Carlsbad, CA Metro Area",30265,NaN,3317200
18,19740,"Denver-Aurora-Lakewood, CO Metro Area",32297,NaN,2851848
20,12580,"Baltimore-Columbia-Towson, MD Metro Area",31209,NaN,2801028


## Using PUMS to impute missing census aggregates

Luckily, we can use the [Public Use Microdata Sample (PUMS)](https://www.census.gov/programs-surveys/acs/technical-documentation/pums.html) to estimate these values ourselves.

[According to the Census](https://www.census.gov/programs-surveys/acs/technical-documentation/pums.html):

>*The American Community Survey (ACS) Public Use Microdata Sample (PUMS) files are a set of untabulated records about individual people or housing units. The Census Bureau produces the PUMS files so that data users can create custom tables that are not available through pretabulated (or summary) ACS data products.*

PUMS are also topcoded, but they're topcoded at $999,999 [CHECK THIS] which should let us get a good estimate of the lower cut-off of the top 5 percent of households.

Getting this PUMS data into a format we can work with takes a long time. Steps 2-4 take an especially long time, so we've cached the results of each of these steps in pickled objects. To create them from scratch, delete or rename the pickles spsecified at the beginning of each step.

### 1. Correlate PUMAs to CBSAs

In order to link the metro areas to the PUMS data, we need to map the GEO.id2 (FIPs codes) to the only geographic type provided in the PUMS data: Public Use Microdata Areas(PUMA). To do this we use the Missouri Census Data Center [Geographic Correspondence Engine](http://mcdc.missouri.edu/websas/geocorr14.html). Let's pull the mappings for Core Based Statistical Areas (CBSA) in all states, because we'll want them later:

![shot1](Screenshots/all_states.png)

We'll be mapping PUMAs to CBSAs (MSAs are a kind of CBSA):

![shot2](Screenshots/mapping.png)

And most PUMAs are entirely within a CBSA, but for those that aren't, Geocorr will estimate the percentage of the PUMA's population contained within that CBSA based off of 2014 measurements:

![shot3](Screenshots/population.png)

Give it a moment to build the file. Then download the CSV, make sure it's named "geocorr14.csv", and put it in the same directory as this notebook. And then...

In [3]:
# ...open it up!
geocorr = pandas.read_csv("geocorr14.csv", encoding = "ISO-8859-1")[1:]
geocorr.head()

,state,puma12,cbsa,stab,cbsaname15,PUMAname,pop14,afact
1,01,00100,,AL,,"Lauderdale, Colbert, Franklin & Marion (Northe...",39326.125,0.21
2,01,00100,22520,AL,"Florence-Muscle Shoals, AL (Metro)","Lauderdale, Colbert, Franklin & Marion (Northe...",147639,0.79
3,01,00200,26620,AL,"Huntsville, AL (Metro)",Limestone & Madison (Outer) Counties--Huntsvil...,183944.849,1
4,01,00301,26620,AL,"Huntsville, AL (Metro)",Huntsville (North) & Madison (East) Cities,124425.297,1
5,01,00302,26620,AL,"Huntsville, AL (Metro)",Huntsville City (Central & South),106218.3,1


PUMAs are unique within states but not between them, so to map them to nationally unique FIPs codes (the 'cbsa' column) we'll need to combine the 'state' and 'puma12' columns into a new field like so:

In [193]:
def make_stpuma(st,puma):
    return int(str(st)+str(puma).rjust(5,'0'))

geocorr['state'] = geocorr.
geocorr['stpuma'] = geocorr.apply(lambda x: make_stpuma(x["state"],x["puma12"]),axis=1)
geocorr.head()

,state,puma12,cbsa,stab,cbsaname15,PUMAname,pop14,afact,stpuma
1,01,00100,NaN,AL,,"Lauderdale, Colbert, Franklin & Marion (Northe...",39326.125,0.21,100100
2,01,00100,22520.0,AL,"Florence-Muscle Shoals, AL (Metro)","Lauderdale, Colbert, Franklin & Marion (Northe...",147639,0.79,100100
3,01,00200,26620.0,AL,"Huntsville, AL (Metro)",Limestone & Madison (Outer) Counties--Huntsvil...,183944.849,1,100200
4,01,00301,26620.0,AL,"Huntsville, AL (Metro)",Huntsville (North) & Madison (East) Cities,124425.297,1,100301
5,01,00302,26620.0,AL,"Huntsville, AL (Metro)",Huntsville City (Central & South),106218.3,1,100302


In [5]:
geocorr["cbsa"] = geocorr["cbsa"].apply(lambda x : pandas.to_numeric(x,errors="coerce"))

#
## Go through the topcoded CBSAs and make a dict for each CBSA. The keys
## are the codes of stpumas that overlap with the CBSA, and the values are 
## Geocorr's estimate of the percentage of the PUMA's population inside 
## the CBSA. E.g. {stpuma: afact} -- we'll need these later
#
def get_pumas(geo_id,geo_corr):
    cbsa = geo_corr[geo_corr["cbsa"] == geo_id] 
    if len(cbsa) == 0:
        print(geo_id)
    target = {}
    for index,row in cbsa.iterrows():
        target[int(row["stpuma"])] = float(row["afact"])
    return target

top_coded["PUMAS"] = top_coded["GEO.id2"].apply(lambda x: get_pumas(x,geocorr))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [6]:
top_coded.head()

,GEO.id2,GEO.display-label,Upper_Limit_Bottom_20,Lower_Limit_Top_5,respop72016,PUMAS
0,35620,"New York-Newark-Jersey City, NY-NJ-PA Metro Area",25717,NaN,20275179,"{3400301: 1.0, 3400302: 1.0, 3400303: 1.0, 340..."
1,31080,"Los Angeles-Long Beach-Anaheim, CA Metro Area",25626,NaN,13328261,"{603701: 1.0, 603702: 1.0, 603703: 1.0, 603704..."
4,26420,"Houston-The Woodlands-Sugar Land, TX Metro Area",25785,NaN,6798010,"{4804400: 1.0, 4804501: 1.0, 4804502: 1.0, 480..."
5,47900,"Washington-Arlington-Alexandria, DC-VA-MD-WV M...",41076,NaN,6150681,"{1100101: 1.0, 1100102: 1.0, 1100103: 1.0, 110..."
9,14460,"Boston-Cambridge-Newton, MA-NH Metro Area",31367,NaN,4805942,"{2500400: 0.352, 2500501: 1.0, 2500502: 1.0, 2..."


### 2. Fetch and parse PUMS data

*This repo has the next few steps of data collection cached, in a file named "pums_we_want.pickle". To do the collection from scratch, delete "pums_we_want.pickle"*

The files we source the PUMS data from are too big for github, but can be downloaded directly from the census:

https://www.census.gov/programs-surveys/acs/data/pums.html

Our code fetches the files we need, puts all the data in a single DataFrame, zips them up and pickles them.

In [149]:
relevant_pumas["state"].unique()

array([ 6,  8,  9, 11, 15, 24, 25, 33, 34, 36, 42, 48, 51, 53, 54])

In [226]:
import zipfile,requests,io,os

#
# This is a table of all the PUMAs we care to look up, and their corresponding
# state fips and CBSA codes. It will throw errors, but they can be ignored. 
#
relevant_pumas = geocorr[geocorr["cbsa"].isin(top_coded["GEO.id2"])]

#
# We need our state and puma codes to be numerical, not strings, in order to
# compare them to the codes in the PUMS data.
#
relevant_pumas["state"] = relevant_pumas["state"].apply(lambda x: pandas.to_numeric(x))
relevant_pumas["puma12"] = relevant_pumas["puma12"].apply(lambda x: pandas.to_numeric(x))

#
# The household Census PUMS for every state in the US are available at this url: 
# https://www2.census.gov/programs-surveys/acs/data/pums/2016/1-Year/csv_hus.zip
# The following code downloads the zip, unzips it, and deletes the irrelevant files.
#
def download_unzip_census(path,directory):
    r = requests.get(path,stream=True)
    zip_ref = zipfile.ZipFile(io.BytesIO(r.content))
    zip_ref.extractall(directory) 

    # Delete any files you don't want
    files = ["ss16husa.csv","ss16husb.csv"]
    for filename in os.listdir(directory):
        if filename not in files:
            os.remove(directory+"/"+filename)
    # [[ Do I need to return something for style points? ]]
    
#
# Take out all states that aren't in our 'relevant' DataFrame, then take out 
# all PUMA codes that aren't synonymous with PUMAs in our 'relevant' DataFrame.
#
# This will leave in some PUMAs in the wrong state, but lets us do our make_stpuma()
# concatenation on a much smaller subset of the data, that still includes all the 
# rows we need. Once we do that concatenation, we can zero in on exactly the 
# rows corresponding to our desired stpumas.
#
def filter_geos(f,relevant):
    f = f[(f["ST"].isin(relevant["state"].unique()))]
    f = f[f["PUMA"].isin(relevant["puma12"].unique())]
    f["stpuma"] = f.apply(lambda x: make_stpuma(x["ST"],x["PUMA"]),axis=1)
    f = f[f["stpuma"].isin(relevant["stpuma"].unique())]
    return f        


#
# If the census files aren't unzipped locally, do that. Then load the contents of 
# both csvs into a single DataFrame
#
def load_pums(directory,relevant):
    if (os.path.isfile("too_big/pums/ss16husa.csv") & os.path.isfile("too_big/pums/ss16husa.csv")):
        download_unzip_census("https://www2.census.gov/programs-surveys/acs/data/pums/2016/1-Year/csv_hus.zip","too_big/pums")
    pums = pandas.DataFrame({})
    for filename in os.listdir(directory):
        if filename[0]!='.':
            tmp = pandas.read_csv(directory+"/"+filename)
            print(tmp.size)
            tmp = filter_geos(tmp,relevant)
            print(tmp.size)
            pums = pandas.concat([pums,tmp])
            print(pums.size)
    return pums

#
# If the pickle exists, fetch that. If not, pull the data again and pickle it. The 
# repo contains the pickle by default, so to rerun this code, delete the pickle or
# or rename it
#
def pums_data(pickle_name,sourcedir,pumas_we_want):
    if (os.path.isfile(pickle_name)):
        return pandas.read_pickle(pickle_name,"gzip")
    target = load_pums(sourcedir,pumas_we_want)
    target.to_pickle(pickle_name,"gzip")
    return target
 
pums_we_want = pums_data("pums_we_want.pickle","too_big/pums/",relevant_pumas)

KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [298]:
pums_we_want.shape

(316676, 231)

### 3. Simulate weighted data

*This step is also cached in a file named "expanded.pickle". To do the processing from scratch, delete "expanded.pickle".*

Now we have all the PUMS data for every geography we care about. This dataframe has a column "WGTP" which indicates how many households each particular record represents. In order to get accurate percentiles, we need each row to represent a single household.

We're going to convert the data by duplicating each row _n_ times, where _n_ equals the row's WGTP.

In [296]:
# !!!!
#
# Actually doing this takes hours and I'm not sure if there's a way I could speed it
# up. Pretty serious obstacle to reproducibility.
#
# I could hit it state by state and write those pickles separately. :shrug: This will
# be too big for github anyways, it won't in individual state files
#
# Oh but accessing by state will be a pain in the butt later. Maybe I'll store them 
# in CBSA files. UHHH BRAIN MELTING.
#
# !!!!
#
# Wait, this might be taking so long because it is broken. Or because iterrows is just
# way slower than apply

# Our PUMS DataFrame has a LOT in it that we don't need,
# so let's shrink it down before processing it

columns_we_want = ["HINCP","WGTP", "stpuma"]
to_expand = pums_we_want[columns_we_want]

# This isn't holding for some reason, keeps reverting to float
#to_expand["WGTP"] = to_expand["WGTP"].astype(int) 


# Make a list of length row[weight_col] where each entry is a dict: {"stpuma":stpuma,"HINCP":income}
def multiply(row, income_col, weight_col, stpuma_col):
    return [{
        stpuma_col: row[stpuma_col],
        income_col: row[income_col]
    } for i in range(int(row[weight_col]))]

# Make a dataframe where each row in our PUMS data is repeated for n rows, where n == row[weight_col]
def explode_weights(df, income_col, weight_col, stpuma_col):
    i =0
    result = []
    for index, row in df.iterrows():
        result = result + multiply(row, income_col, weight_col, stpuma_col)
    df = pandas.DataFrame(result)
    return df

#
# If the pickle exists, fetch that. If not, pull the data again and pickle it. The 
# repo contains the pickle by default, so to rerun this code, delete the pickle or
# or rename it
#
def get_expanded_data(picklename,df,income_col,weight_col,stpuma_col):
    if (os.path.isfile(picklename)):
        return pandas.read_pickle(picklename,"gzip")
    else:
        df[weight_col] = df[weight_col].astype(int)
        
        result = explode_weights(df,income_col,weight_col,stpuma_col)
        result.to_pickle(picklename,"gzip")
        return(result)

expanded = get_expanded_data("expanded.pickle", to_expand, "HINCP", "WGTP", "stpuma")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyboardInterrupt: 

In [ ]:
expanded[expanded["stpuma"]==4200500]

### 4. Map the weighted data to CBSAs based on PUMA allocation factor

*This step is also cached in a file named "cbsa_dict.pickle". To do the collection from scratch, delete "cbsa_dict.pickle".*

We're now going to "sample" all the PUMS data we need, from our new weighted dataset, for each CBSA. If a PUMA has an allocation factor of 1, it's entirely contained within our CBSA, so we take all of the rows in our expanded dataset. If it has an allocation factor less than 1, we randomly sample *n* rows from within our expanded dataset where *n* equals the allocation factor (a percentage) times the total rows in the expanded dataset.

We're doing this in it's own step and storing the results in a dict of the form {CBSA: [HHINC1, ... , HHINCN]}. 

In [ ]:
import pickle, gzip

def write_obj(obj,filename):
    f = gzip.open(filename,'wb')
    pickle.dump(obj,f)
    f.close

def load_obj(filename):
    f=gzip.open(filename,'rb')
    obj = pickle.load(f)
    f.close()
    return(obj)

In [116]:
def get_data(puma,percent,census):
    all_entries = census[census["stpuma"]==puma]
    if percent < 1: 
        print(all_entries.head())
        print(puma)
        s = all_entries.sample(frac=percent,random_state=0)
        return(s)
    return(all_entries)
    
def make_fips_data_dict(df,census,cached_file):
    if (os.path.isfile(cached_file)):
        return load_obj(cached_file)
    else:
        d = {}
        for index, row in df.iterrows():
            tar = pandas.DataFrame(columns=census.columns)
            fip = row["GEO.id2"]
            pumas = row["PUMAS"]
            for p in pumas:
                new = get_data(p,pumas[p],census)
                tar = pandas.concat([tar,new])
            d[fip] = tar
        write_obj(d,cached_file)
        return d
    
fips_data_dict = make_fips_data_dict(top_coded.head(),expanded,"too_big/fips_dict.pickle")

Empty DataFrame
Columns: [HINCP, stpuma]
Index: []
4200500


ValueError: a must be greater than 0